# RABIES installation and usage

Here we specify the code to run RABIES on a test dataset from the StandardRat dataset (subject groups 300800-3008009 and 301400-3001409).

For running rabies preproccessing, default settings were used for all except to following arguments:
--TR 1 --commonspace_reg masking=false,brain_extraction=false,template_registration=SyN,fast_commonspace=true --commonspace_resampling 0.3x0.3x0.3

If based on QC-report this was unstatisfactory, the following arguments were tried iteretiveley: 
1. --anat_autobox --bold_autobox 
2. Option 1 + --anat_inho_cor method=N4_reg,otsu_thresh=2,multiotsu=false --bold_inho_cor method=N4_reg,otsu_thresh=2,multiotsu=false 
3. Option 2 + --bold2anat_coreg masking=false,brain_extraction=false,registration=Rigid

For using RABIES, it is recommended to build an apptainer container. See https://apptainer.org/docs/user/latest/quick_start.html for the instruction to install apptainer. For the latest version of RABIES, see https://hub.docker.com/r/gabdesgreg/rabies/tags

In [ ]:
apptainer pull rabies.sif docker://gabdesgreg/rabies:0.4.8

The following code can be used to run rabies using the default settings. This code will immediately run all three steps, namely preprocessing, confound correction and analysis. For the analysis we have added some of the essentials, such as creating a whole brain functional connectivity matrix and dual-regression independent component analysis 

In [ ]:
# move to working directory
cd $("PATH_TO_WORKING_DIRECTORY")

#specify bids folder
bids="PATH_TO_BIDS_FOLDER"

#specify RABIES container
rabies="PATH_TO_RABIES_CONTAINER"

#specify rat templates and prior ICA maps (RABIES defaults are mouse templates, SIMGA templates are provided)
anat_template="/templates/SIGMA_InVivo_Brain_Template.nii" 
brain_mask="/templates/SIGMA_InVivo_Brain_Mask.nii" 
WM_mask="/templates/SIGMA_InVivo_WM_bin.nii.gz" 
CSF_mask="/templates/SIGMA_InVivo_CSF_bin.nii.gz"
labels="/templates/SIGMA_Anatomical_Brain_Atlas_rs.nii" 
vascular_mask="/templates/SIGMA_InVivo_CSF_bin.nii.gz" 
ICA_prior="/templates/canica_resting_state_clean.nii.gz"

mkdir -p func_preprocess
mkdir -p func_confound
mkdir -p func_analysis

    #preprocess
apptainer run -B $bids:/input_bids:ro -B /func_preprocess:/preprocess_outputs/ $rabies -p MultiProc preprocess /input_bids/ /preprocess_outputs/ --anat_template $anat_template --brain_mask $brain_mask --WM_mask $WM_mask --CSF_mask $CSF_mask --labels $labels --vascular_mask $vascular_mask --TR 1 --commonspace_reg masking=false,brain_extraction=false,template_registration=SyN,fast_commonspace=false --commonspace_resampling 0.3x0.3x0.3 
    
    #confound
apptainer run -B $bids:/input_bids:ro -B /func_preprocess:/preprocess_outputs/ -B /func_confound:/confound_correction_outputs/ $rabies confound_correction /preprocess_outputs/ /confound_correction_outputs/ --TR 1

    #analysis
apptainer run -B $bids:/input_bids:ro -B /func_preprocess:/preprocess_outputs/ -B /func_confound:/confound_correction_outputs/ -B /func_analysis:/analysis_outputs/ $rabies -p MultiProc analysis /confound_correction_outputs/ /analysis_outputs/ --data_diagnosis --prior_maps $ICA_prior --FC_matrix --DR_ICA --group_ica apply=true,dim=10,random_seed=1
